In [ ]:
# this notebook executed on kaggle
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
from tqdm import tqdm

In [ ]:
# Load pre-trained model and tokenizer
model_name = "fathan/indo_sentiment_indobertweet"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/738k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31923, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
model.config.num_labels

2

In [ ]:
df = pd.read_csv("/content/full_text.csv")
texts = df["text"].tolist()

In [ ]:
# Function to predict sentiment
def predict_sentiment(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512,
        padding=True
    )
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        pred_label_id = torch.argmax(probs).item()
        confidence = probs[0][pred_label_id].item()
    return pred_label_id, confidence



# Process all texts and store results
predictions = []
confidences = []

print("Labeling texts...")
for text in tqdm(texts):
    label, conf = predict_sentiment(text)
    predictions.append(label)
    confidences.append(conf)

# Add predictions to your DataFrame
df["predicted_label"] = predictions
df["confidence"] = confidences

Labeling texts...


100%|██████████| 3500/3500 [06:51<00:00,  8.51it/s]


In [ ]:
df.head(20)

,text,predicted_label,confidence
0,Pinjol judol noh urusin...biar ada manfaatnya ...,0,0.997556
1,Percuma 208 juta penduduk dalam usia produktif...,0,0.995568
2,TOMBOL BEGO,0,0.997074
3,Anehnya spam judol ogah nongol dimarih...saket...,0,0.997570
4,"Ok pak generasi muda udah siap, tinggal kabine...",0,0.995840
5,bagus sekali Mas GIbran..\nlebih bagus lagi ji...,1,0.875944
6,"Sebanyak apapun tai di berikan parfum, bau tai...",0,0.998030
7,Tapi boong,0,0.996711
8,Bang syaratnya beliin luna warrior x dong bang,0,0.956415
9,فَمَنْ دَخَلَ عَلَيْهِمْ فَصَدَّقَهُمْ بِكَذِب...,0,0.971004


In [ ]:
df["predicted_label"].value_counts()

,count
predicted_label,
0,3005
1,495


In [ ]:
# Save to a new CSV
df.to_csv("indobertweet_labeled_texts.csv", index=False)
print("Saved to labeled_texts.csv")

Saved to labeled_texts.csv
